banks_projet
exercise from coursera

In [205]:
# import libraries 
import glob 
import xml.etree.ElementTree as ET 
from datetime import datetime 
import sqlite3
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [206]:
#initialization
url= 'https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'
log_file = "code_log.txt"
csv_path='/Users/sophia/Desktop/Dev/project/exchange_rate.csv'
output_path='Largest_banks_data.csv'
table_name='Largest_banks'

In [207]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ':' + message + '\n') 

In [218]:
def extract(url):
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    
    # Find the table containing the data
    table = data.find('table', {'class': 'wikitable'})

    # Extract rows from the table
    rows = table.find_all('tr')
    
    data_list = []

    for row in rows:
        col = row.find_all('td')
        if col:  # Check if there are columns
            rank = col[0].get_text(strip=True) if col[0] else None
            bank_name_title = (col[1].find_all('a')[1])['title'].strip() if col[1].find('a') and col[1].find('a').get('title') else None
            market_cap = col[2].get_text(strip=True) if col[2] else None

            data_dict = {
                "Rank": rank,
                "Bank Name": bank_name_title,
                "MC_USD_Billion": market_cap
            }
            data_list.append(data_dict)

    df = pd.DataFrame(data_list)
    df=df.set_index("Rank")
    df['MC_USD_Billion']=pd.to_numeric(df['MC_USD_Billion'])
    return df



In [220]:
df=extract(url)
df


,Bank Name,MC_USD_Billion
Rank,,
1,JPMorgan Chase,432.92
2,Bank of America,231.52
3,Industrial and Commercial Bank of China,194.56
4,Agricultural Bank of China,160.68
5,HDFC Bank,157.91
6,Wells Fargo,155.87
7,HSBC,148.90
8,Morgan Stanley,140.83
9,China Construction Bank,139.82


In [209]:
def transform(df, csv_path):
    exchange_rate=pd.read_csv(csv_path)
    df['MC_EUR_Billion'] = np.round(df['MC_USD_Billion'] * exchange_rate['Rate'][0], 2)
    df['MC_GBP_Billion'] = np.round(df['MC_USD_Billion'] * exchange_rate['Rate'][1], 2)
    df['MC_INR_Billion'] = np.round(df['MC_USD_Billion'] * exchange_rate['Rate'][2], 2)

    return df

In [221]:
df=transform(df,csv_path)
df

,Bank Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
Rank,,,,,
1,JPMorgan Chase,432.92,402.62,346.34,35910.71
2,Bank of America,231.52,215.31,185.22,19204.58
3,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
4,Agricultural Bank of China,160.68,149.43,128.54,13328.41
5,HDFC Bank,157.91,146.86,126.33,13098.63
6,Wells Fargo,155.87,144.96,124.70,12929.42
7,HSBC,148.90,138.48,119.12,12351.26
8,Morgan Stanley,140.83,130.97,112.66,11681.85
9,China Construction Bank,139.82,130.03,111.86,11598.07


In [210]:
def load_to_csv(df, output_path):
    df.to_csv(output_path) 

In [211]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection,if_exists='replace', index=False)

In [212]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [215]:
#Call fucntions

log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df, csv_path)

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, output_path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('Banks.db')

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')

query_statement = f"SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

sql_connection.close()

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987


In [214]:
df

,Bank Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
Rank,,,,,
1,JPMorgan Chase,432.92,402.62,346.34,35910.71
2,Bank of America,231.52,215.31,185.22,19204.58
3,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
4,Agricultural Bank of China,160.68,149.43,128.54,13328.41
5,HDFC Bank,157.91,146.86,126.33,13098.63
6,Wells Fargo,155.87,144.96,124.70,12929.42
7,HSBC,148.90,138.48,119.12,12351.26
8,Morgan Stanley,140.83,130.97,112.66,11681.85
9,China Construction Bank,139.82,130.03,111.86,11598.07
